In [1]:
import json
def print_json(dict):
    print(json.dumps(dict, sort_keys=True, indent=4))

In [2]:
from chia.consensus.default_constants import DEFAULT_CONSTANTS
from chia.types.blockchain_format.coin import Coin
from chia.types.blockchain_format.program import Program
from chia.types.coin_spend import CoinSpend
from chia.types.condition_opcodes import ConditionOpcode
from chia.util.hash import std_hash
from clvm.casts import int_to_bytes

def get_create_singleton_coin_spends(
        standard_txn_coin, 
        standard_txn_puzzle,
        odd_amount, inner_puzzle, keys_values
    ):

    launcher_coin = Coin(
        standard_txn_coin.name(), 
        singleton_top_layer.SINGLETON_LAUNCHER_HASH, 
        odd_amount
    )
    launcher_id = launcher_coin.name()

    singleton_struct = (
        singleton_top_layer.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer.SINGLETON_LAUNCHER_HASH)
    )
    curried_singleton_puzzle = singleton_top_layer.SINGLETON_MOD.curry(
            singleton_struct,
            inner_puzzle,
    )
    singleton_puzzle_hash = curried_singleton_puzzle.get_tree_hash()

    launcher_solution = Program.to(
        [
            singleton_puzzle_hash,
            int_to_bytes(odd_amount),
            keys_values,
        ]
    )

    launcher_coin_spend = CoinSpend(
            launcher_coin,
            singleton_top_layer.SINGLETON_LAUNCHER,
            launcher_solution
    )
    launcher_announcement = launcher_solution.get_tree_hash()
    standard_txn_coin_conditions = [
        # create launcher coin with the odd_amount (odd)
        Program.to(
            [
                ConditionOpcode.CREATE_COIN,
                singleton_top_layer.SINGLETON_LAUNCHER_HASH,
                odd_amount,
            ]),
        # assert launcher coin announcement
        Program.to(
            [
                ConditionOpcode.ASSERT_COIN_ANNOUNCEMENT, 
                std_hash(launcher_id + launcher_announcement)
            ]),
        # create change minus fee coin (amount - (odd_amount + fee)) to itself
        Program.to(
            [
                ConditionOpcode.CREATE_COIN,
                standard_txn_coin.puzzle_hash,
                standard_txn_coin.amount - odd_amount,
            ]),
    ]
    delegated_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_conditions(standard_txn_coin_conditions)
    solution = p2_delegated_puzzle_or_hidden_puzzle.solution_for_conditions(standard_txn_coin_conditions)
    
    standard_txn_coin_spend = CoinSpend(
        standard_txn_coin,
        standard_txn_puzzle,
        solution
    )
    
    standard_coin_message = (
        delegated_puzzle.get_tree_hash()
        + standard_txn_coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )
    return standard_coin_message, [standard_txn_coin_spend, launcher_coin_spend]

In [3]:
from chia.wallet.puzzles import (singleton_top_layer, p2_delegated_puzzle_or_hidden_puzzle)
from cdv.test import Network, Wallet

network: Network = await Network.create()
await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice) # alice has 2000_000_000_000 mojos

print(f'alice: {alice.balance()}')
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())
standard_coin_message, coin_spends = get_create_singleton_coin_spends(
                    standard_txn_coin,
                    standard_txn_puzzle,
                    odd_amount = 1023, 
                    inner_puzzle = singleton_top_layer.adapt_inner_to_singleton(standard_txn_puzzle), 
                    keys_values = [("Key", "Value")]
              )
standard_txn_coin_spend = coin_spends[0]
launcher_coin_spend = coin_spends[1]
print("\nstarting coin spend:")
print_json(standard_txn_coin_spend.to_json_dict())
print("\nlauncher coin spend:")
print_json(launcher_coin_spend.to_json_dict())
print(f'\nlauncher id: {launcher_coin_spend.coin.name()}')
print(f'\nstandard_coin_message: {standard_coin_message.hex()}')

alice: 2000000000000

starting coin spend:
{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da

In [4]:
from blspy import AugSchemeMPL, PrivateKey
from chia.consensus.default_constants import DEFAULT_CONSTANTS

synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)

signature = AugSchemeMPL.sign(synthetic_sk,standard_coin_message)

In [5]:
from chia.types.spend_bundle import SpendBundle

spend_bundle = SpendBundle(
    coin_spends,
    signature
)
result = await network.push_tx(spend_bundle)
result

{'additions': [Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>, amount=1023),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1749999998977),
  Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: a1af73b7dc0f246ffb0bd41b4ac8f0253f4c0949bfdd070d52a3be037767baa3>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1750000000000),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c06